# Library

In [ ]:
import tensorflow as tf
import zipfile
import os
import random
import csv
import math
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img

# Data Extraction

In [ ]:
# extracting data from zip
local_zip = './data/archive_data/new_dataset.zip'
target_extraction = './data/data_2'
file_check = "ISIC_Labelled"

try:
    if os.path.exists(os.path.join(target_extraction, file_check)):
        print("File Already Extracted")
    else:
        zip_ref = zipfile.ZipFile(local_zip, 'r')
        zip_ref.extractall(target_extraction)
        zip_ref.close()
#if data is already extracted it will instead print file already extracted
except FileNotFoundError:
    print('Zip File Not Found!')

# Check Data and Class

In [ ]:
root = './data/data_2/ISIC_Labelled'
file = []
for files in os.walk(root, topdown=True):
    file.append(files)
    
ffile = file[1:]
file = []
counter = 0
for x in ffile:
    counter += len(x[2])

print(f"there is {counter} data in data_2")
tumor_2 = len(os.listdir(root))
print(f"with {tumor_2} classifications")

# Create Folder

In [ ]:
dir_2 = ['melanoma', 'melanocytic_nevi', 'basal_cell_carcinoma', 'bowen_disease', 'benign_keratosis', 'dermatofibroma', 'vascular_lesions', 'squamous_cell_carcinoma']
root_dir = "./data/final_dataset"
file_check = "Training"
def create_dir(root_path):
    
    try:
        if os.path.exists(os.path.join(root_dir, file_check)):
            for it in os.scandir(root_dir):
                if it.is_dir():
                    print(it.path)
            print("File Already Created!")
        else:
            if os.path.exists(root_dir):
                shutil.rmtree(root_dir)

            os.makedirs(os.path.join(root_path, 'training'))
            os.makedirs(os.path.join(root_path, 'validation'))


            for i in dir_2:
                os.makedirs(os.path.join(f'{root_path}/training', i))
                os.makedirs(os.path.join(f'{root_path}/validation', i))

            print("Folder Created!")
            
    except Exception as e:
        print(f'An error occurred: {str(e)}')

            
create_dir(root_path=root_dir)

In [ ]:
training_paths = []
val_paths = []
for class_name in dir_new:
    training_path = os.path.join(training_main, class_name)
    val_path = os.path.join(validation_main, class_name)
    training_paths.append(training_path)
    val_paths.append(val_path)

In [ ]:
for i, class_path in enumerate (training_paths):
    print(class_path)
print('\n')
for i, class_path in enumerate (val_paths):
    print(class_path)

# Specific Name Change

In [ ]:
path = './data/data_2/ISIC_Labelled'

dir_new = ['melanoma', 'melanocytic_nevi', 'basal_cell_carcinoma', 'bowen_disease', 'benign_keratosis', 'dermatofibroma', 'vascular_lesions', 'squamous_cell_carcinoma']
dir_old = ['Melanoma', 'Melanocytic nevus', 'Basal cell carcinoma', 'Actinic keratosis', 'Benign keratosis', 'Dermatofibroma', 'Vascular lesion', 'Squamous cell carcinoma']

counter = 0

for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    name = os.path.splitext(filename)
    
    for i, x in enumerate(dir_old):
        if x in name:
            postfix = dir_new[i]
            new_name = os.path.join(path, postfix)
            os.rename(file_path, new_name)
            continue
    counter += 1

# Split our Dataset

In [ ]:
counter = 0

for file in (dir_2):
    dir = os.listdir(f"{path}/{file}")
    random.shuffle(dir)
    
    training = dir[:200]
    val = dir[200:239]
    print(f"Training data for {file} is {len(training)}")
    print(f"Validation data for {file} is {len(val)}\n")
    
    root = './data/data_2/ISIC_Labelled/'
    root_dest = './data/final_dataset'
    for i in training:
        source_img = os.path.join(f"{root}/{dir_2[counter]}", i)
        destination = os.path.join(f"{root_dest}/training/{dir_2[counter]}", i)
        copyfile(source_img, destination)
    
    for i in val:
        source_img = os.path.join(f"{root}/{dir_2[counter]}", i)
        destination = os.path.join(f"{root_dest}/validation/{dir_2[counter]}", i)
        copyfile(source_img, destination)
        
    counter+=1

# Set Directory Main

In [ ]:
training_main = './data/final_dataset/training'
validation_main = './data/final_dataset/validation'

# Visualize and Plot the Data

In [ ]:
#show image example
root_path = './data/final_dataset/training/'
fig, axes = plt.subplots(2, 4, figsize=(15,5))
counter = 0
for i in range(2):
    for j in range(4):
        ax = axes[i][j]
        ax.set_title(dir_new[counter])
        ax.imshow(load_img(f"{os.path.join(f'{root_path}/{dir_new[counter]}', os.listdir(f'{root_path+dir_new[counter]}')[random.randint(0,100)])}"))
        ax.set_axis_off()
        plt.show
        counter += 1

# Data Augmentation

In [ ]:
def train_val_generators(training_main, validation_main):
    train_datagen = ImageDataGenerator(
        rescale=1./255.,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    train_generator = train_datagen.flow_from_directory(
        directory=training_main,
        target_size=(150, 150),
        batch_size=64,
        class_mode='categorical'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255.)
    validation_generator = validation_datagen.flow_from_directory(
        directory=validation_main,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = train_val_generators(training_main, validation_main)

# Create Model

In [ ]:
def create_model():
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(8, activation='softmax')
  ])
  

    model.compile(optimizer = tf.optimizers.Adam(),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])       
  
    return model

# Train the Model

In [ ]:
model = create_model()

In [ ]:
model.summary()

In [ ]:


# Train your model
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=100)

# Create Model using VGG16

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

In [ ]:
import numpy as np
vgg = VGG16(include_top=False, input_shape = (150,150,3))

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150,150,3))
base_model.trainable = False ## Not trainable weights

## Preprocessing input
vgg.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
es = EarlyStopping(monitor='accuracy', mode='max', patience=5,  restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, verbose=1)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.90: # @KEEP
                print("\nReached 99% accuracy so cancelling training!") 
                
                # Stop training once the above condition is met
                self.model.stop_training = True
                
callbacks = myCallback()

In [ ]:
import random
tf.random.set_seed(33)
def create_model():
    
    model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAvgPool2D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(62, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
  ])
  

    model.compile(optimizer = tf.optimizers.SGD(learning_rate=0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])       
  
    return model

In [ ]:
history2 = model.fit(train_generator, validation_data=validation_generator, validation_steps=int(0.15*len(validation_generator)),epochs = 50, batch_size = 32, callbacks=[reduce_lr, callbacks])

# Create Model using MobilenetV2

In [ ]:
# tf.random.set_seed(42)
# base_model = tf.keras.applications.MobileNetV2(include_top=False)
# base_model.trainable= False

# inputs = tf.keras.layers.Input(shape=(150,150,3),name = "input_shape",dtype=tf.float16)
# x = base_model(inputs,training=False)
# x = tf.keras.layers.GlobalAvgPool2D(name="global_average_pooling_layer")(x)
# outputs = tf.keras.layers.Dense(8,activation='softmax',dtype=tf.float32,name='outputs')(x)

# model_2 = Model(inputs,outputs)

# model_2.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

# model_2.summary()

In [ ]:
# history2 = model_2.fit(train_generator, validation_data=validation_generator, validation_steps=int(0.15*len(validation_generator)),epochs = 20, batch_size = 32, callbacks=[es])